# Landsat annual composites

1. fetch the Landsat scenes overlapping our study region
2. aggregate the Landsat scenes into a single multi-temporal composite using the median, our composite will contain the median reflectance obtained from all Landsat scenes fetched in certain year.

In [6]:
# we have to install the development version for the time being
#!pip uninstall eodal -y
#!pip install git+https://github.com/lukasValentin/eodal@landsat-dev
#!pip install --upgrade git+https://github.com/EOA-team/eodal
#!pip install --upgrade planetary-computer

In [7]:
# define the year/years for the download

#multiyears
years = list(range(1986, 2023))
excluded_years = [1988, 1992, 1993, 1996, 1997, 2022]
filtered_years = [year for year in years if year not in excluded_years]

#single year
#filtered_years = [1989]

In [8]:
# import libraries
from datetime import datetime
from pathlib import Path
from shapely.geometry import box
from matplotlib import pyplot as plt
import numpy as np

from eodal.config import get_settings
from eodal.core.sensors import Landsat
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from eodal.core.band import GeoInfo, Band
from eodal.core.raster import RasterCollection

In [9]:
# define a function to mask clouds and shadows
def preprocess_landsat_scene(
        ds: Landsat
) -> Landsat:
    """
    Mask clouds and cloud shadows in a Landsat scene based
    on the 'qa_pixel' band.

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
        interfacing libraries!

    :param ds:
        Landsat scene before cloud mask applied.
    :return:
        Landsat scene with clouds and cloud shadows masked.
    """
    ds.mask_clouds_and_shadows(inplace=True)
    return ds

In [10]:
## Setting up the EOdal Mapper
Settings = get_settings()

# we use STAC, i.e., Microsoft Planetary Computer
Settings.USE_STAC = True

# user-inputs
# -------------------------- Collection -------------------------------
collection = 'landsat-c2-l2'

# ---------------------- Spatial Feature  ------------------------------
# can be also shp, gpkg, etc.

# bbox = box(*[30.2825, 0.4019, 30.3714, 0.4643]) # 1. Area
# bbox = box(*[30.2703, 0.4043, 30.4482, 0.5291]) # 2. Area
# bbox = box(*[30.3195, 0.4803, 30.3417, 0.4959]) # 3. Area small for MeanShift trial
# bbox = box(*[30.2433, 0.3746, 30.5640, 0.6962]) # 4. Area bigger
bbox = box(*[30.1, 0.5, 30.6, 0.9]) # 5. North

feature = Feature(
    name='landsat-composite',
    geometry=bbox,
    epsg=4326,
    attributes={})

# ------------------------- Metadata Filters ---------------------------
metadata_filters = [
    #Filter('eo:cloud_cover', '<=', 80),
    #Filter('instruments', '!=', 'etm+')
]

for year in filtered_years: 
    
    # ------------------------- Time Range ---------------------------------
    time_start = datetime(year, 1, 1)
    time_end = datetime(year, 12, 31)

    # set up the Mapper configuration
    mapper_configs = MapperConfigs(
        metadata_filters=metadata_filters,
        collection=collection,
        feature=feature,
        time_start=time_start,
        time_end=time_end)

    # get a new mapper instance
    mapper = Mapper(mapper_configs)

    # fetch the metadata
    # query the scenes available (no I/O of scenes, this only fetches metadata)
    mapper.query_scenes()
    print(str(year), f'Number of Landsat scenes found: {mapper.metadata.shape[0]}')
    
    if mapper.metadata.empty:
        print(f'no scenes found in year: {year}')
        continue

    #We tell EOdal how to load the Landsat scenes using `Landsat.from_usgs`and pass on some kwargs, e.g., the selection of bands we want to read.in addition, we tell EOdal to mask out clouds and shadows and the fly while reading the data using the qa_pixel band (therefore, we set the`read_qa` flag to True.
    
    # define the bands to read in 'band_selection' and to preprocess the scenes
    scene_kwargs = {
        'scene_constructor': Landsat.from_usgs,
        'scene_constructor_kwargs': {'band_selection': ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22'], 'read_qa': True} ,
        'scene_modifier': preprocess_landsat_scene,
        'scene_modifier_kwargs': {}}

    # now we load the scenes
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    # The mapper returns the single scenes. 
    # As we told the EOdal to mask out clouds, a significant share of the pixels is masked out. 
    # We will aggregate them in the next step.
    f = mapper.data.plot(band_selection=['red', 'green', 'blue'], figsize = (20, 20), max_scenes_in_row = 2)

    #save as PNG for quick view
    f.savefig(f'S:\MSc_23_TimckeFinn\data\python_outputs\landsat_scenes_{year}.png')
    
    plt.close(f)  # Close the figure (uses less memory?)

    ## Generating the composite, We will now generate the composite. Here, we have to consider two things: 1. The data is stored as [numpy masked arrays](https://numpy.org/doc/stable/reference/maskedarray.generic.html) as we masked out clouds. 2. We therefore have to use masked arrays and the numpy ma functions that work on numpy masked arrays (i.e., the ignore masked values).

    #First, we open masked arrays for storing the data:
    # all scenes have the same shape, i.e., the same number of bands, rows and columns
    shapes = [{timestamp: scene.get_values().shape} for timestamp, scene in mapper.data]

    # open arrays for storing the data per band
    shape = (len(mapper.data), list(shapes[0].values())[0][1], list(shapes[0].values())[0][2])
    blue = np.ma.masked_array(data=np.ndarray(shape, dtype=float), mask=False)
    red = np.zeros_like(blue)
    green = np.zeros_like(blue)
    nir08 = np.zeros_like(blue)
    swir16 = np.zeros_like(blue)
    swir22 = np.zeros_like(blue)

    # Next, we loop over the scenes.
    idx = 0
    for _, scene in mapper.data:
        blue[idx, :, :] = scene['blue'].values
        red[idx, :, :] = scene['red'].values
        green[idx, :, :] = scene['green'].values
        nir08[idx, :, :] = scene['nir08'].values
        swir16[idx, :, :] = scene['swir16'].values
        swir22[idx, :, :] = scene['swir22'].values
        idx += 1

    # Finally, we aggregate the data using the median reflectance
    # calculate the median reflectance per spectral band
    blue_median = np.ma.median(blue, axis=0)
    green_median = np.ma.median(green, axis=0)
    red_median = np.ma.median(red, axis=0)
    nir08_median = np.ma.median(nir08, axis=0)
    swir16_median = np.ma.median(swir16, axis=0)
    swir22_median = np.ma.median(swir22, axis=0)

    # We store the results in a new RasterCollection save the median reflectance to a new RasterCollection
    rc = RasterCollection()

    bands = {'blue': blue_median, 
            'green': green_median, 
            'red': red_median, 
            'nir08' : nir08_median, 
            'swir16' : swir16_median,
            'swir22' : swir22_median}

    for band_name, band_value in bands.items():
        rc.add_band(
            band_constructor=Band,
            values=band_value,
            band_name=f'{band_name}_median',
            geo_info=scene[band_name].geo_info)
        
    # # We plot the result
    f, ax = plt.subplots(figsize = (20, 10))
    f = rc.plot_multiple_bands(['red_median', 'green_median', 'blue_median'], ax=ax)

    #save as PNG for quick view
    f.savefig(f'S:\MSc_23_TimckeFinn\data\python_outputs\landsat_median_composite_{year}.png')

    plt.close(f)  # Close the figure (uses less memory?)
        
    # save as GeoTiff for further analysis
    rc.to_rasterio(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}.tif')


2023-07-17 16:49:19,590 eodal        INFO     Starting extraction of landsat scenes


1986 Number of Landsat scenes found: 30


2023-07-17 16:49:43,503 eodal        WARNING  Scene with ID LT05_L2SP_173060_19860108_20200918_02_T1 already added to SceneCollection - continue
2023-07-17 16:49:56,139 eodal        WARNING  Scene with ID LT05_L2SP_172060_19860117_20200918_02_T1 already added to SceneCollection - continue
2023-07-17 16:50:20,435 eodal        WARNING  Scene with ID LT05_L2SP_173060_19860124_20200918_02_T2 already added to SceneCollection - continue
2023-07-17 16:50:32,860 eodal        WARNING  Scene with ID LT05_L2SP_172060_19860202_20200918_02_T1 already added to SceneCollection - continue
2023-07-17 16:50:49,628 eodal        WARNING  Scene with ID LT05_L2SP_172060_19860218_20200918_02_T2 already added to SceneCollection - continue
2023-07-17 16:51:28,040 eodal        WARNING  Scene with ID LT05_L2SP_172060_19860712_20200917_02_T1 already added to SceneCollection - continue
2023-07-17 16:51:51,378 eodal        WARNING  Scene with ID LT05_L2SP_172060_19860728_20200918_02_T2 already added to SceneCollect

1987 Number of Landsat scenes found: 28


2023-07-17 16:55:04,374 eodal        WARNING  Scene with ID LT05_L2SP_172060_19870120_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:55:22,238 eodal        WARNING  Scene with ID LT05_L2SP_173060_19870127_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:55:34,239 eodal        WARNING  Scene with ID LT05_L2SP_172060_19870205_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:55:46,920 eodal        WARNING  Scene with ID LT05_L2SP_172060_19870221_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:56:05,819 eodal        WARNING  Scene with ID LT05_L2SP_173060_19870228_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:56:24,398 eodal        WARNING  Scene with ID LT05_L2SP_173060_19870316_20201014_02_T1 already added to SceneCollection - continue
2023-07-17 16:57:28,395 eodal        WARNING  Scene with ID LT05_L2SP_173060_19870807_20201014_02_T1 already added to SceneCollect

...

2016 Number of Landsat scenes found: 176


2023-07-17 20:38:28,915 eodal        WARNING  Scene with ID LE07_L2SP_173060_20160103_20200903_02_T1 already added to SceneCollection - continue
c:\Users\ftimcke\AppData\Local\anaconda3\lib\site-packages\eodal\core\sensors\landsat.py:221: UserWarning: Band cloud_qa not found - remove it from selection
  warnings.warn(
2023-07-17 20:38:39,445 eodal        WARNING  Scene with ID LC08_L2SP_172060_20160104_20200907_02_T1 already added to SceneCollection - continue
2023-07-17 20:39:01,801 eodal        WARNING  Scene with ID LC08_L2SP_173060_20160111_20200907_02_T1 already added to SceneCollection - continue
c:\Users\ftimcke\AppData\Local\anaconda3\lib\site-packages\eodal\core\sensors\landsat.py:221: UserWarning: Band qa_aerosol not found - remove it from selection
  warnings.warn(
2023-07-17 20:39:13,897 eodal        WARNING  Scene with ID LE07_L2SP_172060_20160112_20200903_02_T1 already added to SceneCollection - continue
2023-07-17 20:39:33,927 eodal        WARNING  Scene with ID LE07_L2S

2017 Number of Landsat scenes found: 182


2023-07-17 21:04:47,241 eodal        WARNING  Scene with ID LE07_L2SP_173060_20170105_20200901_02_T1 already added to SceneCollection - continue
c:\Users\ftimcke\AppData\Local\anaconda3\lib\site-packages\eodal\core\sensors\landsat.py:221: UserWarning: Band cloud_qa not found - remove it from selection
  warnings.warn(
2023-07-17 21:04:59,395 eodal        WARNING  Scene with ID LC08_L2SP_172060_20170106_20200905_02_T1 already added to SceneCollection - continue
2023-07-17 21:05:17,224 eodal        WARNING  Scene with ID LC08_L2SP_173060_20170113_20200905_02_T1 already added to SceneCollection - continue
c:\Users\ftimcke\AppData\Local\anaconda3\lib\site-packages\eodal\core\sensors\landsat.py:221: UserWarning: Band qa_aerosol not found - remove it from selection
  warnings.warn(
2023-07-17 21:05:30,259 eodal        WARNING  Scene with ID LE07_L2SP_172060_20170114_20201008_02_T1 already added to SceneCollection - continue
2023-07-17 21:05:46,140 eodal        WARNING  Scene with ID LE07_L2S

STACError: Querying STAC catalog failed: The request exceeded the maximum allowed time, please try again. If the issue persists, please contact planetarycomputer@microsoft.com.

Debug information for support: 0uZa1ZAAAAADdCv8tDaKPR6D4nmoVUTR2WlJIRURHRTA2MTcAOTI3YWJmYTYtMTlmNi00YWYxLWEwOWQtYzk1OWQ5YTFlNjQ0